## This is to store the data frame that will be used for machine learning and check the result for linear regression. The data frame is generated from the cleaned data set 'Property_2019_1_13'. Onehotencoding to categorical features is done to the data set.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Property_2019_1_13.csv')
df.head()

,FID,StreetNumber,StreetName,LandUse,CurrentAcres,TotalGrossArea,FinishedArea,CurrentValue,CurrentLandValue,CurrentYardItemsValue,...,YearBlt,SalePrice,NumofRooms,NumofBedrooms,NumofUnits,ZoningCode,Foundation,Depreciation,PropertyCenterPoint_x,PropertyCenterPoint_y
0,1004,69,32,5176,0.05597,1760,840.00000,152500.0,121600,0,...,1912,158000,5,2,1,1733,2628,0.359,-73.2122,44.46910
1,1005,25,21,5176,0.08242,2676,1436.00000,241900.0,94400,700,...,1899,231750,7,3,1,1733,2628,0.098,-73.2125,44.48854
2,1006,133,137,2150,0.00000,2184,1572.80005,268000.0,0,0,...,1986,381500,6,2,1,5042,1219,0.025,-73.2681,44.52157
3,1007,14,10,5176,0.09832,3699,2087.00000,340800.0,130100,0,...,1910,430000,8,4,1,5042,2628,0.174,-73.2051,44.48153
4,1009,237,499,367,0.70154,62852,37977.00000,1905600.0,547700,51800,...,2013,505683,103,53,26,1733,1219,0.000,-73.2258,44.48770


In [3]:
len(df)

4308

In [4]:
df.columns

Index(['FID', 'StreetNumber', 'StreetName', 'LandUse', 'CurrentAcres',
       'TotalGrossArea', 'FinishedArea', 'CurrentValue', 'CurrentLandValue',
       'CurrentYardItemsValue', 'CurrentBuildingValue', 'BuildingType',
       'HeatFuel', 'HeatType', 'Grade', 'YearBlt', 'SalePrice', 'NumofRooms',
       'NumofBedrooms', 'NumofUnits', 'ZoningCode', 'Foundation',
       'Depreciation', 'PropertyCenterPoint_x', 'PropertyCenterPoint_y'],
      dtype='object')

### A preliminary feature selection for machine learning is as follows.
#### 1.FID should not be used in any type of analysis. 
#### 2.Since StreetName, StreetNumber, ZoningCode are all location variables which are less significant in determining SalePrice than the coordinate variables, as seen in Statistical Analysis, we will not use them as features in machine learning. However, we will preserve PropertyCenterPoint_x and PropertyCenterPoint_y.
#### 3.It has been found in the data cleaning step that CurrentValue = CurrentLandValue+CurrentYardItemsValue+CurrentBuildingValue. CurrentValue and SalePrice is linearly related. We will preserve SalePrice, CurrentLandValue, CurrentYardItemsValue, CurrentBuildingValue but ignore CurrentValue in machine learning algorithms.
#### 4.Recall TotalGrossArea and FinishedArea are strongly linearly related, and that SalePrice depends more on FinishedArea than on TotalGrossArea based on the correlation matrix, we will use FinishedArea instead of TotalGrossArea as a feature in machine learning algorithms.
#### 5.Recall NumofRooms and NumofBedrooms are strongly linearly related, and that SalePrice depends more on NumofRooms than on NumofBedrooms based on the correlation matrix, we will use NumofRooms instead of NumofBedrooms as a feature in machine learning algorithms.

In [5]:
df_feature = df[['LandUse','CurrentAcres', 'TotalGrossArea', 'FinishedArea', 
                 'CurrentLandValue', 'CurrentYardItemsValue', 'CurrentBuildingValue',
                 'BuildingType', 'HeatFuel', 'HeatType', 'Grade', 'YearBlt', 'SalePrice',
                 'NumofRooms', 'NumofUnits', 'Foundation',
                 'Depreciation', 'PropertyCenterPoint_x', 'PropertyCenterPoint_y']]
df_feature.head()

,LandUse,CurrentAcres,TotalGrossArea,FinishedArea,CurrentLandValue,CurrentYardItemsValue,CurrentBuildingValue,BuildingType,HeatFuel,HeatType,Grade,YearBlt,SalePrice,NumofRooms,NumofUnits,Foundation,Depreciation,PropertyCenterPoint_x,PropertyCenterPoint_y
0,5176,0.05597,1760,840.00000,121600,0,30900.0,2168,1039,3893,5,1912,158000,5,1,2628,0.359,-73.2122,44.46910
1,5176,0.08242,2676,1436.00000,94400,700,146800.0,83,8187,4255,7,1899,231750,7,1,2628,0.098,-73.2125,44.48854
2,2150,0.00000,2184,1572.80005,0,0,268000.0,482,8187,4255,10,1986,381500,6,1,1219,0.025,-73.2681,44.52157
3,5176,0.09832,3699,2087.00000,130100,0,210700.0,2168,8187,3893,8,1910,430000,8,1,2628,0.174,-73.2051,44.48153
4,367,0.70154,62852,37977.00000,547700,51800,1306100.0,291,8187,4255,9,2013,505683,103,26,1219,0.000,-73.2258,44.48770


### The columns that orginally contain categorical data will be transformed by OneHotEncoding for better machine learning results.

#### Encode LandUse

In [6]:
LandUseCoded = pd.get_dummies(df_feature.LandUse,prefix='LandUse', drop_first=True)
df_feature = pd.concat([df_feature,LandUseCoded],axis=1)
df_feature.drop(['LandUse'],axis=1, inplace=True)
df_feature.columns

Index(['CurrentAcres', 'TotalGrossArea', 'FinishedArea', 'CurrentLandValue',
       'CurrentYardItemsValue', 'CurrentBuildingValue', 'BuildingType',
       'HeatFuel', 'HeatType', 'Grade', 'YearBlt', 'SalePrice', 'NumofRooms',
       'NumofUnits', 'Foundation', 'Depreciation', 'PropertyCenterPoint_x',
       'PropertyCenterPoint_y', 'LandUse_8', 'LandUse_43', 'LandUse_45',
       'LandUse_117', 'LandUse_239', 'LandUse_285', 'LandUse_367',
       'LandUse_369', 'LandUse_1041', 'LandUse_2150', 'LandUse_5176'],
      dtype='object')

#### Encode BuildingType

In [7]:
BuildingTypeCoded = pd.get_dummies(df_feature.BuildingType,prefix='BuildingType', drop_first=True)
df_feature = pd.concat([df_feature,BuildingTypeCoded],axis=1)
df_feature.drop(['BuildingType'],axis=1, inplace=True)
df_feature.columns

Index(['CurrentAcres', 'TotalGrossArea', 'FinishedArea', 'CurrentLandValue',
       'CurrentYardItemsValue', 'CurrentBuildingValue', 'HeatFuel', 'HeatType',
       'Grade', 'YearBlt', 'SalePrice', 'NumofRooms', 'NumofUnits',
       'Foundation', 'Depreciation', 'PropertyCenterPoint_x',
       'PropertyCenterPoint_y', 'LandUse_8', 'LandUse_43', 'LandUse_45',
       'LandUse_117', 'LandUse_239', 'LandUse_285', 'LandUse_367',
       'LandUse_369', 'LandUse_1041', 'LandUse_2150', 'LandUse_5176',
       'BuildingType_8', 'BuildingType_9', 'BuildingType_11',
       'BuildingType_12', 'BuildingType_14', 'BuildingType_22',
       'BuildingType_24', 'BuildingType_30', 'BuildingType_31',
       'BuildingType_39', 'BuildingType_44', 'BuildingType_46',
       'BuildingType_60', 'BuildingType_81', 'BuildingType_83',
       'BuildingType_87', 'BuildingType_101', 'BuildingType_103',
       'BuildingType_108', 'BuildingType_110', 'BuildingType_111',
       'BuildingType_144', 'BuildingType_147', 'Buil

#### Encode HeatFuel

In [8]:
HeatFuelCoded = pd.get_dummies(df_feature.HeatFuel,prefix='HeatFuel', drop_first=True)
df_feature = pd.concat([df_feature,HeatFuelCoded],axis=1)
df_feature.drop(['HeatFuel'],axis=1, inplace=True)
df_feature.columns

Index(['CurrentAcres', 'TotalGrossArea', 'FinishedArea', 'CurrentLandValue',
       'CurrentYardItemsValue', 'CurrentBuildingValue', 'HeatType', 'Grade',
       'YearBlt', 'SalePrice', 'NumofRooms', 'NumofUnits', 'Foundation',
       'Depreciation', 'PropertyCenterPoint_x', 'PropertyCenterPoint_y',
       'LandUse_8', 'LandUse_43', 'LandUse_45', 'LandUse_117', 'LandUse_239',
       'LandUse_285', 'LandUse_367', 'LandUse_369', 'LandUse_1041',
       'LandUse_2150', 'LandUse_5176', 'BuildingType_8', 'BuildingType_9',
       'BuildingType_11', 'BuildingType_12', 'BuildingType_14',
       'BuildingType_22', 'BuildingType_24', 'BuildingType_30',
       'BuildingType_31', 'BuildingType_39', 'BuildingType_44',
       'BuildingType_46', 'BuildingType_60', 'BuildingType_81',
       'BuildingType_83', 'BuildingType_87', 'BuildingType_101',
       'BuildingType_103', 'BuildingType_108', 'BuildingType_110',
       'BuildingType_111', 'BuildingType_144', 'BuildingType_147',
       'BuildingType_162

#### Encode HeatType

In [9]:
HeatTypeCoded = pd.get_dummies(df_feature.HeatType,prefix='HeatType', drop_first=True)
df_feature = pd.concat([df_feature,HeatTypeCoded],axis=1)
df_feature.drop(['HeatType'],axis=1, inplace=True)
df_feature.columns

Index(['CurrentAcres', 'TotalGrossArea', 'FinishedArea', 'CurrentLandValue',
       'CurrentYardItemsValue', 'CurrentBuildingValue', 'Grade', 'YearBlt',
       'SalePrice', 'NumofRooms', 'NumofUnits', 'Foundation', 'Depreciation',
       'PropertyCenterPoint_x', 'PropertyCenterPoint_y', 'LandUse_8',
       'LandUse_43', 'LandUse_45', 'LandUse_117', 'LandUse_239', 'LandUse_285',
       'LandUse_367', 'LandUse_369', 'LandUse_1041', 'LandUse_2150',
       'LandUse_5176', 'BuildingType_8', 'BuildingType_9', 'BuildingType_11',
       'BuildingType_12', 'BuildingType_14', 'BuildingType_22',
       'BuildingType_24', 'BuildingType_30', 'BuildingType_31',
       'BuildingType_39', 'BuildingType_44', 'BuildingType_46',
       'BuildingType_60', 'BuildingType_81', 'BuildingType_83',
       'BuildingType_87', 'BuildingType_101', 'BuildingType_103',
       'BuildingType_108', 'BuildingType_110', 'BuildingType_111',
       'BuildingType_144', 'BuildingType_147', 'BuildingType_162',
       'Building

#### Encode Foundation

In [10]:
FoundationCoded = pd.get_dummies(df_feature.Foundation,prefix='Foundation', drop_first=True)
df_feature = pd.concat([df_feature,FoundationCoded],axis=1)
df_feature.drop(['Foundation'],axis=1, inplace=True)
df_feature.columns

Index(['CurrentAcres', 'TotalGrossArea', 'FinishedArea', 'CurrentLandValue',
       'CurrentYardItemsValue', 'CurrentBuildingValue', 'Grade', 'YearBlt',
       'SalePrice', 'NumofRooms', 'NumofUnits', 'Depreciation',
       'PropertyCenterPoint_x', 'PropertyCenterPoint_y', 'LandUse_8',
       'LandUse_43', 'LandUse_45', 'LandUse_117', 'LandUse_239', 'LandUse_285',
       'LandUse_367', 'LandUse_369', 'LandUse_1041', 'LandUse_2150',
       'LandUse_5176', 'BuildingType_8', 'BuildingType_9', 'BuildingType_11',
       'BuildingType_12', 'BuildingType_14', 'BuildingType_22',
       'BuildingType_24', 'BuildingType_30', 'BuildingType_31',
       'BuildingType_39', 'BuildingType_44', 'BuildingType_46',
       'BuildingType_60', 'BuildingType_81', 'BuildingType_83',
       'BuildingType_87', 'BuildingType_101', 'BuildingType_103',
       'BuildingType_108', 'BuildingType_110', 'BuildingType_111',
       'BuildingType_144', 'BuildingType_147', 'BuildingType_162',
       'BuildingType_219', 'Bu

In [11]:
df_feature.to_csv('Ready_for_machine_learning_2019_1_23.csv',index=False)

#### Set the target and variables.

In [12]:
X = df_feature.drop(columns=['SalePrice'])
y = df_feature.SalePrice

#### Generate Train and Test sets, and perform scaling.

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Linear Regression

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
reg = LinearRegression()
reg.fit(X_train, y_train)
mse = mean_squared_error(y_test, reg.predict(X_test))
print("MSE: %.4f" % mse)
print("Square of MSE: %.4f" % np.sqrt(mse))
print("Average Percentage of Abosulute Error: %.4f" % np.mean(np.abs(reg.predict(X_test)-y_test)/y_test))

MSE: 13544726393.2428
Square of MSE: 116381.8130
Average Percentage of Abosulute Error: 0.2331


##### Compared to the best model in Gradient Boosting which has MSE = 9653647785.6226 and the corresponding average percentage of absolute error = 20.18%, Linear Regression is not a good model. Next we will try to establish a simplified linear regression model by only taking a few important features. The features selected are in the top list of the most important feature in the Gradient Boosting Regression.

In [16]:
X2=df_feature[['BuildingType_291', 'BuildingType_11', 'HeatType_54',
       'BuildingType_8', 'Grade', 'LandUse_285', 'BuildingType_144',
       'CurrentYardItemsValue', 'BuildingType_298', 'LandUse_117',
       'PropertyCenterPoint_y', 'Foundation_2322', 'NumofRooms',
       'PropertyCenterPoint_x', 'Depreciation', 'YearBlt', 'CurrentAcres',
       'TotalGrossArea', 'CurrentLandValue', 'FinishedArea', 'NumofUnits',
       'CurrentBuildingValue']]
y2=df_feature.SalePrice

from sklearn.model_selection import train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=0)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)

from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X2_train, y2_train)
mse = mean_squared_error(y2_test, reg.predict(X2_test))
print("MSE: %.4f" % mse)
print("Square of MSE: %.4f" % np.sqrt(mse))
print("Average Percentage of Abosulute Error: %.4f" % np.mean(np.abs(reg.predict(X2_test)-y2_test)/y2_test))

MSE: 13124780312.9596
Square of MSE: 114563.4336
Average Percentage of Abosulute Error: 0.2232


##### Indeed, the linear regression performs better but is still not as good as the Gradient Boosting Regression.